In [1]:
#Import essential libraries for the initial data processing
import pandas as pd
import numpy as np

In [2]:
#The code below reads my CSV file
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_59b31594f5cc4c1ea7cc2d997f2a6077 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='jWYMftPiAiF-SYoymEY4A6SyuJp6bJb5E21g91yAZ5KX',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_59b31594f5cc4c1ea7cc2d997f2a6077.get_object(Bucket='courseracapstone-donotdelete-pr-mcqilmhdsobnfo',Key='City of Sydney .csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df= pd.read_csv(body)
df.head()

,Postcode,Council,Suburb,Unnamed: 3
0,2015,City of Sydney,Alexandria,NaN
1,2038,City of Sydney,Annandale,NaN
2,2000,City of Sydney,Barangaroo,NaN
3,2015,City of Sydney,Beaconsfield,NaN
4,2050,City of Sydney,Camperdown,NaN


The dataframe above contains an unnamed column with non-existent values. The first thing we will do is discard this column using the .drop command.

In [3]:
df.drop(['Unnamed: 3'], axis=1, inplace=True)
df.head()

,Postcode,Council,Suburb
0,2015,City of Sydney,Alexandria
1,2038,City of Sydney,Annandale
2,2000,City of Sydney,Barangaroo
3,2015,City of Sydney,Beaconsfield
4,2050,City of Sydney,Camperdown


Let's now explore our postcodes and merge suburbs in a duplicated postcode as a single row.

In [4]:
df['Postcode'].value_counts()

2000    6
2011    4
2021    3
2015    3
2010    2
2037    2
2008    2
2017    2
2044    1
2007    1
2009    1
2016    1
2043    1
2018    1
2038    1
2042    1
2050    1
Name: Postcode, dtype: int64

The following postcodes have multiple suburbs under a similar postal code: 2000, 2011, 2021, 2015, 2010, 2037, 2008, 2017. We will use a groupby function to combine those rows into one.

In [5]:
df = df.groupby(['Postcode','Council'])['Suburb'].apply(', '.join).reset_index()
df.head()

,Postcode,Council,Suburb
0,2000,City of Sydney,"Barangaroo, Dawes Point, Haymarket, Millers Po..."
1,2007,City of Sydney,Ultimo
2,2008,City of Sydney,"Chippendale, Darlington"
3,2009,City of Sydney,Pyrmont
4,2010,City of Sydney,"Darlinghurst, Surry Hills"


Let's double check our group by command worked and that now each postcode only appears once.

In [6]:
df['Postcode'].value_counts()

2015    1
2037    1
2017    1
2018    1
2021    1
2042    1
2000    1
2050    1
2038    1
2043    1
2007    1
2008    1
2009    1
2010    1
2011    1
2044    1
2016    1
Name: Postcode, dtype: int64

Exactly what we needed! Now let's check the shape of our databases before start looking for the geographic coordinates for it.

In [7]:
df.shape

(17, 3)

We currently have 16 different postcodes in the City of Sydney Council, as the information above include the headers in the count.

The next step before working with maps for venues exploration is adding a Latitude and Longitude column into our dataframe.I have used a query system called <b>pgeocode</b> to obtain the Latitude and Longitude of the postal codes in the City of Sydney. To ensure the original data was not modified, I created a new dataframe to display the result. 

In [8]:
#Let's install the package
! pip install pgeocode

In [9]:
#import required library
import pgeocode

In [10]:
#the code below applies pgeocode to Australia and allows us to obtain the postalcodes in a new dataframe.
geo = pgeocode.Nominatim('au')
df1= geo.query_postal_code(["2000", "2007", "2008", "2009", "2010", "2011", "2015", "2016", "2017", "2018", "2021", "2037", "2038", "2042", "2043", "2044", "2050"])

In [11]:
df1.head()

,postal_code,country_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,2000,AU,"Barangaroo, Millers Point, Haymarket, Parliame...",New South Wales,NSW,SYDNEY STREETS,NaN,NaN,NaN,-33.8641,151.204013,4.0
1,2007,AU,"Ultimo, Broadway",New South Wales,NSW,SYDNEY STREETS,NaN,NaN,NaN,-33.8806,151.200100,4.0
2,2008,AU,"Darlington, Chippendale",New South Wales,NSW,SYDNEY STREETS,NaN,NaN,NaN,-33.8907,151.196400,4.0
3,2009,AU,Pyrmont,New South Wales,NSW,SYDNEY STREETS,NaN,NaN,NaN,-33.8703,151.194300,4.0
4,2010,AU,"Surry Hills, Darlinghurst",New South Wales,NSW,SYDNEY STREETS,NaN,NaN,NaN,-33.8846,151.215350,4.0


In [12]:
#Applied the pd.concat command to merge the dataframes df and df1 with an inner joint. 
dfgeo = pd.concat([df, df1], axis=1, join='inner')
dfgeo.head()

,Postcode,Council,Suburb,postal_code,country_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,2000,City of Sydney,"Barangaroo, Dawes Point, Haymarket, Millers Po...",2000,AU,"Barangaroo, Millers Point, Haymarket, Parliame...",New South Wales,NSW,SYDNEY STREETS,NaN,NaN,NaN,-33.8641,151.204013,4.0
1,2007,City of Sydney,Ultimo,2007,AU,"Ultimo, Broadway",New South Wales,NSW,SYDNEY STREETS,NaN,NaN,NaN,-33.8806,151.200100,4.0
2,2008,City of Sydney,"Chippendale, Darlington",2008,AU,"Darlington, Chippendale",New South Wales,NSW,SYDNEY STREETS,NaN,NaN,NaN,-33.8907,151.196400,4.0
3,2009,City of Sydney,Pyrmont,2009,AU,Pyrmont,New South Wales,NSW,SYDNEY STREETS,NaN,NaN,NaN,-33.8703,151.194300,4.0
4,2010,City of Sydney,"Darlinghurst, Surry Hills",2010,AU,"Surry Hills, Darlinghurst",New South Wales,NSW,SYDNEY STREETS,NaN,NaN,NaN,-33.8846,151.215350,4.0


In [13]:
#Dropped all columns not required as part of the data cleaning proccess. 
dfgeo.drop(['postal_code','country_code','place_name','state_name','state_code','county_name','county_code','community_name','community_code','accuracy'], axis=1, inplace=True)
dfgeo.head()

,Postcode,Council,Suburb,latitude,longitude
0,2000,City of Sydney,"Barangaroo, Dawes Point, Haymarket, Millers Po...",-33.8641,151.204013
1,2007,City of Sydney,Ultimo,-33.8806,151.200100
2,2008,City of Sydney,"Chippendale, Darlington",-33.8907,151.196400
3,2009,City of Sydney,Pyrmont,-33.8703,151.194300
4,2010,City of Sydney,"Darlinghurst, Surry Hills",-33.8846,151.215350


In [14]:
#Rename both latitude and longitude column to add capital letters and keep the consistence on the names in the dataframe dfgeo. 
dfgeo.rename(columns={'latitude':'Latitude','longitude':'Longitude'}, inplace=True)
dfgeo.head()

,Postcode,Council,Suburb,Latitude,Longitude
0,2000,City of Sydney,"Barangaroo, Dawes Point, Haymarket, Millers Po...",-33.8641,151.204013
1,2007,City of Sydney,Ultimo,-33.8806,151.200100
2,2008,City of Sydney,"Chippendale, Darlington",-33.8907,151.196400
3,2009,City of Sydney,Pyrmont,-33.8703,151.194300
4,2010,City of Sydney,"Darlinghurst, Surry Hills",-33.8846,151.215350


In [15]:
dfgeo.shape

(17, 5)

Now, we have all of our latitude and longitude data we are ready to explore each of the suburbs and explore further the City of Sydney Area, but let's first download all libraries and functions we may require: 

In [16]:
import requests # library to handle requests
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 81kB 9.4MB/s  eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
Folium installed
Libraries imported.


In [17]:
#Define my Foursquare information
CLIENT_ID = 'RWXZLBMYPCN4NXJRLTNQNI45MT2V1IOS3JNIYAB1DIFTNNKO' # your Foursquare ID
CLIENT_SECRET = '5OGQBTH2H5QUWTK5CYRPL5BZ2QS5M32X3ZTSKNXHY3RS0EIX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RWXZLBMYPCN4NXJRLTNQNI45MT2V1IOS3JNIYAB1DIFTNNKO
CLIENT_SECRET:5OGQBTH2H5QUWTK5CYRPL5BZ2QS5M32X3ZTSKNXHY3RS0EIX


<h3> Identifying Venues in the City of Sydney Area</h3>

With all libraries installed, now I will generate the Geographical Coordinates for the City of Sydney and plot the neighbourhoods part of the area in a map to make sure our geographical coordinates are correct.

In [18]:
#Used Geopy to get the Geographical Coordinates for the City of Sydney: 
address = 'City of Sydney, NSW'
geolocator = Nominatim(user_agent="citysyd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of the City of Sydney area are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of the City of Sydney area are -33.8853222, 151.2065221.


In [38]:
#Let's visualize the neighbourhoods in the area of City of Sydney:

#Create map of City of Sydney using latitude and longitude values
map_dfgeo = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfgeo['Latitude'], dfgeo['Longitude'], dfgeo['Suburb']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dfgeo)  
    
map_dfgeo

The map seems to be correctly reflecting the central suburbs that are withing the central business district and the City of Sydney Area. 

As this project is looking for venues where we can set up an activation to promote an alcoholic drink, we will search for venues under the category <b>Nightlife</b>, and then if required narrow our search down. 

In [20]:
#determine search paramethers
section = 'topPicks'
categoryId = '4d4b7105d754a06376d81259'
query = 'Drinks'
radius = 5000
time= 'any'
day= 'any'
sortByPopularity = 1

In [27]:
#Define url to be used in query: 
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&section={}&categoryId={}&query={}&radius={}&time{}&day{}&limit={}&sortByPopularity{}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, section, categoryId, query, radius, time, day, LIMIT, sortByPopularity)
url

'https://api.foursquare.com/v2/venues/explore?client_id=RWXZLBMYPCN4NXJRLTNQNI45MT2V1IOS3JNIYAB1DIFTNNKO&client_secret=5OGQBTH2H5QUWTK5CYRPL5BZ2QS5M32X3ZTSKNXHY3RS0EIX&ll=-33.8853222,151.2065221&v=20180604&section=topPicks&categoryId=4d4b7105d754a06376d81259&query=Drinks&radius=5000&timeany&dayany&limit=100&sortByPopularity1'

In [28]:
#Send Get request for results:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa4d0537fff58258531f620'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Sydney',
  'headerFullLocation': 'Sydney',
  'headerLocationGranularity': 'city',
  'query': 'drinks',
  'totalResults': 148,
  'suggestedBounds': {'ne': {'lat': -33.84032215499995,
    'lng': 151.26062767939246},
   'sw': {'lat': -33.93032224500004, 'lng': 151.15241652060755}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b29ebacf964a5201ea424e3',
       'name': 'Royal Albert Hotel',
       'location': {'address': '140 Commonwealth St',
        'crossStreet': 'at Reservoir St',
        'lat': -33.88133152419853,
        'lng': 151.2108539413369,
        'labeledLatL

Foursquares allow you to either search or explore. The <b>explore</b> function allows you to explore an area, while the <b>search</b> allows you to search for venues that have been tagged wtih specific paramethers. 

In order  to narrow down the search to the type of venues we required, I also applied to the link a series of search paramethers, including the ones listed below: 

<ul>
    <li> section: Foursquare allows you to select from food, drinks, coffee, shops, arts, outdoors, sights, trending, nextVenues (venues frequently visited after a given venue), or topPicks (a mix of recommendations generated without a query from the user). </li>
    <li> categoryId: The one I used is specific for the category <b> NIghtlife Spots</b>. You can apply multiple category ID's to your search and it helps you to limite the results to an specific category.</li>
    <li> query: A query uses a term to be searched against a venue's category, without any effect on the section (if specified). </li>
    <li> radius: Radius to search within, in meters.</li>
<li>time: This is one of  the main things for me. I figured out that since I was looking for a venue that is mostly open after-hours, but trying to run my searches during day times, I wasn't able to visualise many results, therefore it's important to set up your time to <b>'any'</b> - meaning you are asking the application to retrieve results for any time of day!</li>
    <li> day: Similar to the above, if you are trying to search for venues on a Monday and have not set this parameter, your result will only show information for venues open on Mondays. I have set the search to show <b>'any'</b>, meaning the command will retrieve results for any day of the week. 
<li> sortByPopularity: The last of the parameters I set to limit my search was the sortByPopularity, what this does is sort the results by popularity instead of relevance.</li>
    </ul>

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = results['response']['groups'][0]['items']
    
SYD_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.city', 'venue.location.postalcode']
SYD_venues = SYD_venues.loc[:, filtered_columns]

# filter the category for each row
SYD_venues['venue.categories'] = SYD_venues.apply(get_category_type, axis=1)

# clean columns
SYD_venues.columns = [col.split(".")[-1] for col in SYD_venues.columns]

SYD_venues.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,name,categories,lat,lng,city,postalcode
0,Royal Albert Hotel,Pub,-33.881332,151.210854,Surry Hills,NaN
1,Opera Bar,Cocktail Bar,-33.858409,151.213976,Sydney,NaN
2,Handpicked Cellar Door,Wine Bar,-33.885712,151.201842,Chippendale,NaN
3,The Wine Library,Wine Bar,-33.888961,151.232513,Woollahra,NaN
4,The Village Inn,Pub,-33.883514,151.222550,Paddington,NaN


Let's check how many venues have been returned by FourSquares.

In [31]:
print('{} venues were returned by Foursquare.'.format(SYD_venues.shape[0]))

100 venues were returned by Foursquare.


When you check the top lines of the results of our search you can see that there was 148 responses, but as we have we have limited our search to 100 venues, that is the results the dataframe is returning. Also note that we are looking into venues sorted by Popularity and that is the sequence the venues have enteresd our dataframe for SYD Venues. 

I will drop the postalcode colunm for the dataframe created. 

In [32]:
#Drop column postalcode: 
SYD_venues.drop(['postalcode'], axis =1, inplace= True)
SYD_venues.head()

,name,categories,lat,lng,city
0,Royal Albert Hotel,Pub,-33.881332,151.210854,Surry Hills
1,Opera Bar,Cocktail Bar,-33.858409,151.213976,Sydney
2,Handpicked Cellar Door,Wine Bar,-33.885712,151.201842,Chippendale
3,The Wine Library,Wine Bar,-33.888961,151.232513,Woollahra
4,The Village Inn,Pub,-33.883514,151.222550,Paddington


We have already plotted a map of the City of Sydney Area, showing all of the suburbs on it with blue markers. Let's now plot a Venues Map, so show the venues in the area, so we can observe how they are spread. 

In [41]:
# add the venues as green circle markers to the map of the City of Sydney. 
for lat, lng, label in zip(SYD_venues.lat, SYD_venues.lng, SYD_venues.categories):
    folium.CircleMarker(
    [lat, lng],
    radius=4,
    color='green',
    popup=label,
    fill = True,
    fill_color='green',
    fill_opacity=0.6
    ).add_to(map_dfgeo)

# display map
map_dfgeo


As we have to make a recommendation based on the top 30 venues, I will provide the client with the map above, so he knows his top 100 pick, but I will also generate a new dataframe with only the top 30 venues (remember that our search has already done this job for us on sorting the venues by popularity). 

In [45]:
Top30_SYD = SYD_venues[:31]
Top30_SYD.head()

,name,categories,lat,lng,city
0,Royal Albert Hotel,Pub,-33.881332,151.210854,Surry Hills
1,Opera Bar,Cocktail Bar,-33.858409,151.213976,Sydney
2,Handpicked Cellar Door,Wine Bar,-33.885712,151.201842,Chippendale
3,The Wine Library,Wine Bar,-33.888961,151.232513,Woollahra
4,The Village Inn,Pub,-33.883514,151.222550,Paddington


In [46]:
Top30_SYD.shape

(31, 5)

In [47]:
Top30_SYD

,name,categories,lat,lng,city
0,Royal Albert Hotel,Pub,-33.881332,151.210854,Surry Hills
1,Opera Bar,Cocktail Bar,-33.858409,151.213976,Sydney
2,Handpicked Cellar Door,Wine Bar,-33.885712,151.201842,Chippendale
3,The Wine Library,Wine Bar,-33.888961,151.232513,Woollahra
4,The Village Inn,Pub,-33.883514,151.222550,Paddington
5,Gallon,Bar,-33.869517,151.193928,Pyrmont
6,The Glenmore Rooftop Hotel,Pub,-33.858632,151.207464,The Rocks
7,Palisade Hotel,Pub,-33.857979,151.202264,Sydney
8,Continental Deli Bar Bistro,Deli / Bodega,-33.896342,151.178397,Newtown
9,Lord Nelson Brewery Hotel,Brewery,-33.858403,151.203548,Millers Point


Now let's plot a map with the recommended venues only. I will again plot on top of the original map with the neighbourhoods in blue, all venues in green and our top venues will be identified in <span style="color: red">red</span>. 

In [50]:
#Create a second map for the City of Sydney: 

City_of_Sydney = folium.Map(location=[latitude, longitude], zoom_start=13)

# add the Top 30 venues as red circle markers to the map of the City of Sydney. 
for lat, lng, label in zip(Top30_SYD['lat'], Top30_SYD['lng'], Top30_SYD['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.6,
        parse_html=False).add_to(City_of_Sydney)  

#Display Map    
City_of_Sydney

I will also add the neighbourhoods back to the second map with the blue marks, so we can visualize if we are allocating well the venues. 

In [51]:
# add suburbs to the map of recommendations
for lat, lng, label in zip(dfgeo['Latitude'], dfgeo['Longitude'], dfgeo['Suburb']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(City_of_Sydney)  
    
City_of_Sydney